In [1]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image
from googletrans import Translator
from gtts import gTTS
import os
from flask import Flask, render_template, request, send_file

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)
    
    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)
    
    output_ids = model.generate(pixel_values, **gen_kwargs)
    
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

app = Flask(__name__)

def translate_text(text, dest_language):
    translator = Translator()
    translated_text = translator.translate(text, dest=dest_language)
    return translated_text.text

def text_to_speech(text, language):
    tts = gTTS(text=text, lang=language, slow=False)
    tts.save("static/caption_audio.mp3")

@app.route('/', methods=['GET'])
def index():
    return render_template('main.html')

@app.route('/', methods=['POST'])
def predict():
    # Get the uploaded image file
    imagefile = request.files['imagefile']
    # Save the image file
    image_path = './static/' + imagefile.filename
    imagefile.save(image_path)
    # Get the selected language from the form
    selected_language = request.form['language']
    # Generate caption for the uploaded image
    caption = predict_step([image_path])
    if caption:
        caption_text = caption[0]  # Extract the string from the list
    else:
        caption_text = "No caption generated."

    # Translate the generated caption
    translated_caption = translate_text(caption_text, selected_language)
    # Convert caption to speech
    text_to_speech(translated_caption, selected_language)
    return render_template('display3.html', prediction=translated_caption, path=image_path)

# Route to serve the audio file
@app.route('/caption_audio')
def caption_audio():
    return send_file("static/caption_audio.mp3", as_attachment=True)


/Users/bharathmogalapu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bharathmogalapu/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/bharathmogalapu/Library/Python/3.9/lib/python/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:6969
Press CTRL+C to quit
127.0.0.1 - - [05/Aug/2024 16:56:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2024 16:56:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2024 16:56:18] "GET /main.css HTTP/1.1" 404 -
127.0.0.1 - - [05/Aug/2024 16:56:18] "GET /static/image.png HTTP/1.1" 200 -
